### Set Up

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import re

In [ ]:
# BENCHMARK_DIR = "data/benchmarks/"
BENCHMARK_DIR = "data/sample_cnf/"

# If notebook working dir is `notebooks/`, resolve BENCHMARK_DIR relative to repo root
if not os.path.isabs(BENCHMARK_DIR):
    repo_root_candidate = os.path.abspath(os.path.join(os.getcwd(), '..'))
    candidate = os.path.join(repo_root_candidate, BENCHMARK_DIR)
    if os.path.exists(candidate):
        BENCHMARK_DIR = candidate
    else:
        BENCHMARK_DIR = os.path.abspath(BENCHMARK_DIR)

OUTPUT_CSV = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), "data", "features", "instance_features.csv")
print('Resolved BENCHMARK_DIR =', BENCHMARK_DIR)
print('Will write OUTPUT_CSV =', OUTPUT_CSV)


In [ ]:
# Verify BENCHMARK_DIR and find .cnf files
print('BENCHMARK_DIR =', repr(BENCHMARK_DIR))
abs_dir = os.path.abspath(BENCHMARK_DIR)
print('Absolute path =', abs_dir)
pattern = os.path.join(BENCHMARK_DIR, '**', '*.cnf')
matched = glob.glob(pattern, recursive=True)
print(f'Found {len(matched)} .cnf files with pattern: {pattern}')
for p in matched[:20]:
    print(' -', p)

print('Current working dir =', os.getcwd())


### Parse CNF header

In [ ]:
def parse_cnf_header(cnf_path):
    with open(cnf_path, 'r', errors='ignore') as f:
        for line in f:
            if line.startswith('p cnf'):
                parts = line.split()
                # p cnf <num_vars> <num_clauses>
                n = int(parts[2])
                m = int(parts[3])
                return n, m
    return None, None

In [ ]:
rows = []
# search recursively for .cnf files under BENCHMARK_DIR
for cnf_path in glob.glob(os.path.join(BENCHMARK_DIR, "**", "*.cnf"), recursive=True):
    fname = os.path.basename(cnf_path)
    n, m = parse_cnf_header(cnf_path)
    if n is None or m is None:
        print(f"Warning: cannot parse header for {fname}")
        continue
    ratio = m / n if n>0 else np.nan
    
    # Additional features: e.g., Horn clause fraction, literal balance
    horn_count = 0      # number of Horn clauses
    total_clauses = 0   # total number of clauses
    pos_lits = 0        # total positive literals
    neg_lits = 0        # total negative literals
    clause_sizes = []
    with open(cnf_path, 'r', errors='ignore') as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith('c') or line.startswith('p'):
                continue
            parts = line.split()
            if not parts:
                continue
            lits = []
            for x in parts:
                if x == '0':
                    break
                if re.match(r'^-?\d+$', x):
                    lits.append(int(x))
                else:
                    # skip any non-integer token safely
                    continue

            if not lits:
                continue
            total_clauses += 1
            clause_sizes.append(len(lits))
            pos = sum(1 for l in lits if l>0)
            neg = sum(1 for l in lits if l<0)
            pos_lits += pos
            neg_lits += neg
            # Horn clause: at most one positive literal
            if pos <= 1:
                horn_count += 1

    horn_frac = horn_count / total_clauses if total_clauses>0 else np.nan
    pos_neg_balance = pos_lits / neg_lits if neg_lits>0 else np.nan
    clause_size_mean = np.mean(clause_sizes) if clause_sizes else np.nan
    clause_size_max = np.max(clause_sizes) if clause_sizes else np.nan
    
    rows.append({
        "instance": fname,
        "num_vars": n,
        "num_clauses": m,
        "clause_var_ratio": ratio,
        "horn_frac": horn_frac,
        "pos_neg_balance": pos_neg_balance,
        "clause_size_mean": clause_size_mean,
        "clause_size_max": clause_size_max
    })


df = pd.DataFrame(rows)
if not df.empty and 'instance' in df.columns:
    df.set_index("instance", inplace=True)
    display(df.head())
else:
    print("No instances were parsed — DataFrame is empty.")


In [ ]:
# Save feature
out_dir = os.path.dirname(OUTPUT_CSV)
if out_dir and not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)

if 'df' in globals() and not df.empty:
    df.to_csv(OUTPUT_CSV)
    print(f"Saved features to {OUTPUT_CSV}")
else:
    print("No features to save (DataFrame missing or empty).")


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Histogram: clause/variable ratio
plt.hist(df["clause_var_ratio"].dropna(), bins=30)
plt.title("Distribution of clause/variable ratio")
plt.xlabel("m/n")
plt.ylabel("Number of instances")
plt.show()

# Scatter plot: horn_frac vs clause_var_ratio
plt.scatter(df["horn_frac"], df["clause_var_ratio"])
plt.xlabel("Horn clause fraction")
plt.ylabel("Clause/var ratio")
plt.title("Horn frac vs clause/var ratio")
plt.show()

# Scatter plot: num_vars vs clause_var_ratio
plt.figure(figsize=(6,4))
plt.scatter(df["num_vars"], df["clause_var_ratio"], alpha=0.7)
plt.xlabel("Number of variables (n)")
plt.ylabel("Clause/variable ratio (m/n)")
plt.title("n vs m/n")
plt.grid(True)
plt.show()

# Colour-code by instance size category (small vs large)
size_thresh = df["num_vars"].median()
plt.figure(figsize=(6,4))
colors = ["blue" if n <= size_thresh else "red" for n in df["num_vars"]]
plt.scatter(df["num_vars"], df["horn_frac"], c=colors, alpha=0.7)
plt.xlabel("Number of variables (n)")
plt.ylabel("Horn clause fraction")
plt.title("n vs horn_frac (blue=small, red=large)")
plt.grid(True)
plt.show()

# Descriptive stats by size category
small_df = df[df["num_vars"] <= size_thresh]
large_df = df[df["num_vars"] > size_thresh]
print("Small instances (<= {} vars):".format(size_thresh))
print(small_df.describe())
print("\nLarge instances (>= {} vars):".format(size_thresh))
print(large_df.describe())


